In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler

# resample the dataset
from sklearn.utils import resample
from sklearn.utils import shuffle

# import various functions from sklearn
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier


# import the XGBoost function for classification
from xgboost import XGBClassifier

import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [4]:
data = pd.read_csv('/content/drive/MyDrive/CAPSTONE/CAPSTONE_PROJECT/0_dataset/3.FEATURED_ENGINEERING DATASET/Encoded_Data.csv')

In [5]:
df=data.copy()

In [6]:
df.head(3)

,Unnamed: 0,Road Name,Cross-Street Name,Driver Substance Abuse,Driver At Fault,Injury Severity,Driver Distracted By,Drivers License State,Vehicle Body Type,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Latitude,Longitude,Vehicle Model,hour,Crash_year,Crash_month,Crash_day,Crash_hour,Crash_week,Agency Name_Maryland_National_Capital,Agency Name_Montgomery_County_Police,Agency Name_Rockville_Police_Department,Agency Name_Takoma_Park_Police_Department,ACRS Report Type_Injury Crash,ACRS Report Type_Property Damage Crash,Route Type_County_Route,Route Type_Crossover,Route Type_Government_Route,Route Type_Interstate_Route,Route Type_Local_Route,Route Type_Maryland_State_Route,Route Type_Municipality_Route,Route Type_Other_Public_Roadway,Route Type_Private_Route,Route Type_Ramp_Route,Route Type_Service_Road,Route Type_Spur_Route,Route Type_US_State_Route,Collision Type_Angle + Left Turn,Collision Type_Angle + Right Turn,Collision Type_Angle Collision,Collision Type_Back-to-Back,Collision Type_Both Left Turns (Opposite Direction),Collision Type_Both Left Turns (Same Direction),Collision Type_Head-On,Collision Type_Head-On + Left Turn,Collision Type_Left Turn (Same Direction),Collision Type_Other,Collision Type_Rear vs Side,Collision Type_Rear-End (Same Direction),Collision Type_Right Turn (Same Direction),Collision Type_Sideswipe (Opposite Direction),Collision Type_Sideswipe (Same Direction),Collision Type_Single Vehicle,Circumstance_Category_Congestion / Backup,Circumstance_Category_Driver Inattention,Circumstance_Category_Lane Violation,Circumstance_Category_Mechanical / Environmental,Circumstance_Category_Passing / Turning,Circumstance_Category_Right-of-Way Violation,Circumstance_Category_Road Obstruction,Circumstance_Category_Speed Related,Circumstance_Category_Surface Condition,Circumstance_Category_Tailgating,Circumstance_Category_Traffic Control Violation,Circumstance_Category_Weather Related,Weather_Blowing Snow,Weather_Clear,Weather_Cloudy,Weather_Fog,Weather_Freezing Rain,Weather_Other,Weather_Rain,Weather_Severe Crosswinds,Weather_Severe Winds,Weather_Sleet,Weather_Snow,Weather_Wintry Mix,Surface Condition_Ice,Surface Condition_Mud_Dirt_Gravel,Surface Condition_Oil,Surface Condition_Other,Surface Condition_Sand,Surface Condition_Slush,Surface Condition_Snow,Surface Condition_Wet,Light_Dark - Not Lighted,Light_Dark - Unknown Lighting,Light_Dawn,Light_Daylight,Light_Dusk,Light_Other,Traffic Control_Flashing Traffic Signal,Traffic Control_Lane Use Control,Traffic Control_No Control,Traffic Control_Other,Traffic Control_Other Pavement Marking,Traffic Control_Other Signal,Traffic Control_Pedestrian Crossing,Traffic Control_Person Control,Traffic Control_Railroad Crossing Device,Traffic Control_Railroad Crossing Signal,Traffic Control_Ramp Meter Signal,Traffic Control_School Zone Sign,Traffic Control_Stop Sign,Traffic Control_Traffic Signal,Traffic Control_Warning Sign,Traffic Control_Yield Sign,Vehicle Damage Extent_Disabling,Vehicle Damage Extent_Functional,Vehicle Damage Extent_No Damage,Vehicle Damage Extent_Other,Vehicle Damage Extent_Superficial,Vehicle Damage Extent_Vehicle Not at Scene,Vehicle First Impact Location_Eight O Clock,Vehicle First Impact Location_Eleven O Clock,Vehicle First Impact Location_Five O Clock,Vehicle First Impact Location_Four O Clock,Vehicle First Impact Location_Nine O Clock,Vehicle First Impact Location_NonCollision,Vehicle First Impact Location_One O Clock,Vehicle First Impact Location_RoofTop,Vehicle First Impact Location_Seven O Clock,Vehicle First Impact Location_Six O Clock,Vehicle First Impact Location_Ten O Clock,Vehicle First Impact Location_Three O Clock,Vehicle First Impact Location_Twelve O Clock,Vehicle First Impact Location_Two O Clock,Vehicle First Impact Location_Underside,Vehicle First Impact Location_VehicleNotAtScene,Vehicle Movement_Backing,Vehicle Movement_ChangingLanes,Vehicle Movement_DriverlessMovingVehicle,Vehicle Movement_EnteringTrafficLane,Vehicl

In [7]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [8]:
df.shape

(205539, 168)

## Model-1: Master + Validation Split and Train–Test Split
### Step 1: Load Encoded Dataset
We load the complete encoded dataset which will be used to create splits.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

target = "Driver Distracted By"
X = df.drop(columns=[target])
y = df[target]

print("Dataset loaded:", df.shape)


Dataset loaded: (205539, 168)


## Step 2: Create Master (90%) and Validation (10%) Split
Master dataset is used for model training and testing.  
Validation dataset is for final unbiased evaluation.


In [10]:
X_major, X_val, y_major, y_val = train_test_split(
    X, y,
    test_size=0.10,
    stratify=y,
    random_state=42
)

major_df = pd.concat([X_major, y_major], axis=1)
val_df   = pd.concat([X_val, y_val], axis=1)

print("Master dataset:", major_df.shape)
print("Validation dataset:", val_df.shape)


Master dataset: (184985, 168)
Validation dataset: (20554, 168)


## Step 3: Save Master and Validation Files
Both files are saved in the Model-1 folder for later use.


In [11]:
save_path = "/content/drive/MyDrive/CAPSTONE/CAPSTONE_PROJECT/08_MODELS/Model7_Distraction_Cause/"

major_df.to_csv(save_path + "model_7_master_data.csv", index=False)
val_df.to_csv(save_path + "model_7_validation_data.csv", index=False)

print("Saved model_7_master_data.csv and model_7_validation_data.csv")


Saved model_7_master_data.csv and model_7_validation_data.csv


In [12]:
df_master_data = pd.read_csv(save_path + "model_7_master_data.csv")


# IMPORTANT  USER DEFINED FUNCTION

### UNIVERSAL MODEL FUNCTION

In [13]:
# ================================================================
# UNIVERSAL MODEL FUNCTION (simple + clear scaling logic)
# ================================================================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

def run_model(model, X, y, test_size=0.20, scaled=False, threshold=0.5):
    """
    scaled = True  → apply scaling on numeric columns
    scaled = False → no scaling
    model  = sklearn model OR "stats" for statsmodels logistic
    """

    # 1) Train–Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )

    # 2) Scaling if selected
    if scaled:
        scaler = StandardScaler()
        num_cols = X.select_dtypes(include='number').columns

        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_test[num_cols]  = scaler.transform(X_test[num_cols])

    # 3) Statsmodels Logit
    if model == "stats":
        X_train_c = sm.add_constant(X_train)
        X_test_c  = sm.add_constant(X_test)

        logit = sm.Logit(y_train, X_train_c).fit(disp=False)

        yproba_train = logit.predict(X_train_c)
        yproba_test  = logit.predict(X_test_c)

        ypred_train = (yproba_train >= threshold).astype(int)
        ypred_test  = (yproba_test >= threshold).astype(int)

        return logit, X_train, X_test, y_train, y_test, ypred_train, ypred_test, yproba_train, yproba_test

    # 4) Normal sklearn model
    model.fit(X_train, y_train)

    ypred_train = model.predict(X_train)
    ypred_test  = model.predict(X_test)

    yproba_train = model.predict_proba(X_train)[:,1]
    yproba_test  = model.predict_proba(X_test)[:,1]

    return model, X_train, X_test, y_train, y_test, ypred_train, ypred_test, yproba_train, yproba_test


### METRICS FUNCTION

In [14]:
# ================================================================
# METRICS FUNCTION (Train + Test separate + Binary/Multiclass safe)
# ================================================================

import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score,
    f1_score, roc_auc_score, cohen_kappa_score,
    classification_report, confusion_matrix
)

d = pd.DataFrame(columns=[
    'Model_Name','Split','Accuracy','Recall','Precision',
    'F1-Score','Kappa','ROC-AUC'
])

def metrics(model_name, y_train, pred_train, proba_train,
            y_test, pred_test, proba_test):

    global d

    # Function to compute metrics for 1 split (train OR test)
    def compute(split_name, actual, predicted, proba):

        unique_classes = len(pd.Series(actual).unique())
        is_binary = (unique_classes == 2)
        avg = "binary" if is_binary else "weighted"

        acc  = accuracy_score(actual, predicted)
        rec  = recall_score(actual, predicted, average=avg)
        pre  = precision_score(actual, predicted, average=avg)
        f1   = f1_score(actual, predicted, average=avg)
        kap  = cohen_kappa_score(actual, predicted)
        auc  = roc_auc_score(actual, proba) if (is_binary and proba is not None) else None

        # append to global dataframe
        d.loc[len(d)] = [model_name, split_name, acc, rec, pre, f1, kap, auc]

        # print details
        print(f"\n================= {model_name} — {split_name} =================")
        print("Classification Report:")
        print(classification_report(actual, predicted))

        print("Confusion Matrix:")
        print(confusion_matrix(actual, predicted))

        if auc is not None:
            print("ROC-AUC:", auc)

    # ---- TRAIN METRICS ----
    compute("Train", y_train, pred_train, proba_train)

    # ---- TEST METRICS ----
    compute("Test", y_test, pred_test, proba_test)

    return d

### Feature Importance Function

In [15]:
def fi(model, x, n_features=10):
    df_fi = pd.DataFrame({
        "Feature": x.columns,
        "Importance": model.feature_importances_
    })
    return df_fi.sort_values(by="Importance", ascending=False).head(n_features)


### ROC Curve

In [16]:
def plot_roc_plain(y_test, yproba_test):
    fpr, tpr, _ = roc_curve(y_test, yproba_test)
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1])
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title("ROC Curve")
    plt.show()


# MODEL BUILDING AND TRAINING


### Logit Model (Statsmodels Logistic Regression)

###  Why Statsmodels Logit Failed

- Statsmodels **Logit requires a binary target** (only 0 and 1).
- Our Injury Severity column has **5 classes** (0, 1, 2, 3, 4).
- Because the target is **multi-class**, Logit cannot estimate probabilities in the 0–1 range.
- Therefore, Logit throws the error: *“endog must be in the unit interval.”*


## Logistic Regression (Sklearn) — Model Call
This model supports multi-class Injury Severity and works correctly without converting to binary.


In [17]:
X = df_master_data.drop(columns=target)
y = df_master_data[target]

In [18]:
from sklearn.linear_model import LogisticRegression

# Define model
log_reg = LogisticRegression(multi_class="multinomial")

# Run model using universal function
log_reg, X_train_lr, X_test_lr, y_train_lr, y_test_lr, ypred_train_lr, ypred_test_lr, yproba_train_lr, yproba_test_lr = run_model(
    model=log_reg,
    X=X,
    y=y,
    test_size=0.20,
    scaled=True
)

# Correct metrics call
metrics(
    model_name="LogisticRegression",
    y_train=y_train_lr,
    pred_train=ypred_train_lr,
    proba_train=yproba_train_lr,
    y_test=y_test_lr,
    pred_test=ypred_test_lr,
    proba_test=yproba_test_lr
)


================= LogisticRegression — Train =================
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       107
           1       0.00      0.00      0.00        39
           2       0.00      0.00      0.00        33
           3       0.00      0.00      0.00       292
           4       0.00      0.00      0.00       277
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00       154
           7       0.00      0.00      0.00        50
           8       0.40      0.00      0.00      4254
           9       0.56      0.56      0.56     35312
          10       0.00      0.00      0.00       170
          11       0.54      0.03      0.06       221
          12       0.80      0.88      0.84    102286
          13       0.43      0.00      0.00      3291
          14       0.00      0.00      0.00       258
          15       0.00      0.00      0.00     

,Model_Name,Split,Accuracy,Recall,Precision,F1-Score,Kappa,ROC-AUC
0,LogisticRegression,Train,0.744500,0.744500,0.710032,0.715132,0.387239,None
1,LogisticRegression,Test,0.739925,0.739925,0.683393,0.710220,0.374652,None


## Decision Tree Classifier — Model Call
Simple non-linear classifier, works well without scaling.


In [19]:
from sklearn.tree import DecisionTreeClassifier

# Define model
dt = DecisionTreeClassifier(random_state=42)

# Run model using universal function
dt, X_train_dt, X_test_dt, y_train_dt, y_test_dt, ypred_train_dt, ypred_test_dt, yproba_train_dt, yproba_test_dt = run_model(
    model=dt,
    X=X,
    y=y,
    test_size=0.20,
    scaled=False    # Trees do NOT need scaling
)

# Correct metrics function call
metrics(
    model_name="DecisionTree",
    y_train=y_train_dt,
    pred_train=ypred_train_dt,
    proba_train=yproba_train_dt,
    y_test=y_test_dt,
    pred_test=ypred_test_dt,
    proba_test=yproba_test_dt
)


================= DecisionTree — Train =================
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       107
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00       292
           4       1.00      1.00      1.00       277
           5       1.00      1.00      1.00        50
           6       1.00      1.00      1.00       154
           7       1.00      1.00      1.00        50
           8       1.00      1.00      1.00      4254
           9       1.00      1.00      1.00     35312
          10       1.00      1.00      1.00       170
          11       1.00      1.00      1.00       221
          12       1.00      1.00      1.00    102286
          13       1.00      1.00      1.00      3291
          14       1.00      1.00      1.00       258
          15       1.00      1.00      1.00       330


,Model_Name,Split,Accuracy,Recall,Precision,F1-Score,Kappa,ROC-AUC
0,LogisticRegression,Train,0.744500,0.744500,0.710032,0.715132,0.387239,None
1,LogisticRegression,Test,0.739925,0.739925,0.683393,0.710220,0.374652,None
2,DecisionTree,Train,1.000000,1.000000,1.000000,1.000000,1.000000,None
3,DecisionTree,Test,0.660567,0.660567,0.665630,0.663070,0.275250,None


## Random Forest Classifier — Model Call
Ensemble of decision trees, robust to imbalance and noise.


In [20]:
from sklearn.ensemble import RandomForestClassifier

# Define model
rf = RandomForestClassifier(random_state=42)

# Run model using universal function
rf, X_train_rf, X_test_rf, y_train_rf, y_test_rf, ypred_train_rf, ypred_test_rf, yproba_train_rf, yproba_test_rf = run_model(
    model=rf,
    X=X,
    y=y,
    test_size=0.20,
    scaled=False     # Trees & ensemble trees do NOT need scaling
)

# Correct metrics function call
metrics(
    model_name="RandomForest",
    y_train=y_train_rf,
    pred_train=ypred_train_rf,
    proba_train=yproba_train_rf,
    y_test=y_test_rf,
    pred_test=ypred_test_rf,
    proba_test=yproba_test_rf
)


================= RandomForest — Train =================
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       107
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00       292
           4       1.00      1.00      1.00       277
           5       1.00      1.00      1.00        50
           6       1.00      1.00      1.00       154
           7       1.00      1.00      1.00        50
           8       1.00      1.00      1.00      4254
           9       1.00      1.00      1.00     35312
          10       1.00      1.00      1.00       170
          11       1.00      1.00      1.00       221
          12       1.00      1.00      1.00    102286
          13       1.00      1.00      1.00      3291
          14       1.00      1.00      1.00       258
          15       1.00      1.00      1.00       330


,Model_Name,Split,Accuracy,Recall,Precision,F1-Score,Kappa,ROC-AUC
0,LogisticRegression,Train,0.744500,0.744500,0.710032,0.715132,0.387239,None
1,LogisticRegression,Test,0.739925,0.739925,0.683393,0.710220,0.374652,None
2,DecisionTree,Train,1.000000,1.000000,1.000000,1.000000,1.000000,None
3,DecisionTree,Test,0.660567,0.660567,0.665630,0.663070,0.275250,None
4,RandomForest,Train,0.999980,0.999980,0.999980,0.999980,0.999956,None
5,RandomForest,Test,0.751818,0.751818,0.719714,0.718727,0.388553,None


## AdaBoost Classifier — Model Call
Boosting model good for imbalanced classes.


In [22]:
from sklearn.ensemble import AdaBoostClassifier

# Define model
ada = AdaBoostClassifier(random_state=42)

# Run universal model function
ada, X_train_ada, X_test_ada, y_train_ada, y_test_ada, ypred_train_ada, ypred_test_ada, yproba_train_ada, yproba_test_ada = run_model(
    model=ada,
    X=X,
    y=y,
    test_size=0.20,
    scaled=False   # Tree-based → NO scaling needed
)

# Correct metrics call (train + test)
metrics(
    model_name="AdaBoost",
    y_train=y_train_ada,
    pred_train=ypred_train_ada,
    proba_train=yproba_train_ada,
    y_test=y_test_ada,
    pred_test=ypred_test_ada,
    proba_test=yproba_test_ada
)


================= AdaBoost — Train =================
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       107
           1       0.00      0.00      0.00        39
           2       0.00      0.00      0.00        33
           3       0.00      0.00      0.00       292
           4       0.00      0.00      0.00       277
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00       154
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00      4254
           9       0.51      0.33      0.40     35312
          10       0.00      0.00      0.00       170
          11       0.00      0.00      0.00       221
          12       0.75      0.91      0.82    102286
          13       0.00      0.00      0.00      3291
          14       0.00      0.00      0.00       258
          15       0.00      0.00      0.00       330
    

,Model_Name,Split,Accuracy,Recall,Precision,F1-Score,Kappa,ROC-AUC
0,LogisticRegression,Train,0.744500,0.744500,0.710032,0.715132,0.387239,None
1,LogisticRegression,Test,0.739925,0.739925,0.683393,0.710220,0.374652,None
2,DecisionTree,Train,1.000000,1.000000,1.000000,1.000000,1.000000,None
3,DecisionTree,Test,0.660567,0.660567,0.665630,0.663070,0.275250,None
4,RandomForest,Train,0.999980,0.999980,0.999980,0.999980,0.999956,None
5,RandomForest,Test,0.751818,0.751818,0.719714,0.718727,0.388553,None
6,AdaBoost,Train,0.708814,0.708814,0.636213,0.662228,0.231377,None
7,AdaBoost,Test,0.709571,0.709571,0.637119,0.662980,0.233104,None


## XGBoost Classifier — Model Call
High-performance boosting model; does not require scaling.


In [23]:
from xgboost import XGBClassifier

# Define model
xgb = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss"
)

# Run model
xgb, X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb, ypred_train_xgb, ypred_test_xgb, yproba_train_xgb, yproba_test_xgb = run_model(
    model=xgb,
    X=X,
    y=y,
    test_size=0.20,
    scaled=False   # XGBoost does NOT require scaling
)

# CORRECT metrics call
metrics(
    model_name="XGBoost",
    y_train=y_train_xgb,
    pred_train=ypred_train_xgb,
    proba_train=yproba_train_xgb,
    y_test=y_test_xgb,
    pred_test=ypred_test_xgb,
    proba_test=yproba_test_xgb
)


================= XGBoost — Train =================
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.55      0.71       107
           1       1.00      1.00      1.00        39
           2       1.00      1.00      1.00        33
           3       1.00      0.25      0.40       292
           4       1.00      0.31      0.48       277
           5       1.00      0.98      0.99        50
           6       1.00      0.60      0.75       154
           7       1.00      1.00      1.00        50
           8       0.86      0.13      0.23      4254
           9       0.66      0.65      0.65     35312
          10       0.99      0.41      0.58       170
          11       1.00      0.60      0.75       221
          12       0.84      0.91      0.88    102286
          13       0.99      0.06      0.11      3291
          14       1.00      0.28      0.44       258
          15       1.00      0.28      0.44       330
     

,Model_Name,Split,Accuracy,Recall,Precision,F1-Score,Kappa,ROC-AUC
0,LogisticRegression,Train,0.744500,0.744500,0.710032,0.715132,0.387239,None
1,LogisticRegression,Test,0.739925,0.739925,0.683393,0.710220,0.374652,None
2,DecisionTree,Train,1.000000,1.000000,1.000000,1.000000,1.000000,None
3,DecisionTree,Test,0.660567,0.660567,0.665630,0.663070,0.275250,None
4,RandomForest,Train,0.999980,0.999980,0.999980,0.999980,0.999956,None
5,RandomForest,Test,0.751818,0.751818,0.719714,0.718727,0.388553,None
6,AdaBoost,Train,0.708814,0.708814,0.636213,0.662228,0.231377,None
7,AdaBoost,Test,0.709571,0.709571,0.637119,0.662980,0.233104,None
8,XGBoost,Train,0.798862,0.798862,0.804147,0.780071,0.526735,None
9,XGBoost,Test,0.753926,0.753926,0.712481,0.726219,0.410988,None


# Full Model Comparison & Inference Summary — Driver Distraction Classification (Model 7)

---

## Table 1 — Performance Summary (Train vs Test)

| Model             | Train Acc | Test Acc | Gap (Train−Test) | Overfitting Status       | Business Interpretation |
|------------------|-----------|----------|------------------|---------------------------|-------------------------|
| Logistic Regression | 0.744    | 0.740    | 0.004            | ❌ No Overfitting         | Stable baseline; interpretable but moderate accuracy. |
| Decision Tree       | 1.000    | 0.661    | 0.339            | ⚠️ Severe Overfitting     | Memorizes training data; unreliable for real-world use. |
| Random Forest       | 1.000    | 0.752    | 0.248            | ⚠️ Strong Overfitting     | Good test accuracy but clearly overfits; needs tuning. |
| AdaBoost            | 0.709    | 0.710    | 0.001            | ❌ No Overfitting         | Very stable but accuracy lower than RF/XGB. |
| XGBoost             | 0.799    | 0.754    | 0.045            | ⚠️ Mild Overfitting       | Best performance in this group; strong candidate. |

---

## Table 2 — Overfitting / Underfitting Check

| Model               | Status              | Reason |
|--------------------|----------------------|--------|
| Logistic Regression | Balanced             | Train ≈ Test, stable. |
| Decision Tree       | Severe Overfitting   | Train = 100%, test accuracy drops sharply. |
| Random Forest       | Strong Overfitting   | Train = 100%, test far lower. |
| AdaBoost            | Balanced             | Train ≈ Test; no variance. |
| XGBoost             | Mild Overfitting     | Slight accuracy drop but still strong. |

---

## Table 3 — Business Interpretation

| Model               | Business Impact | Best Use Case |
|--------------------|-----------------|----------------|
| Logistic Regression | Reliable & explainable | Use when interpretability is required (e.g., policy or legal reports). |
| Decision Tree       | Too unstable | Not suitable for deployment. |
| Random Forest       | High potential after tuning | Use only after overfitting reduction (pruning, limiting depth). |
| AdaBoost            | Very stable but weaker | Use for low-risk systems requiring consistent behavior. |
| XGBoost             | Best accuracy overall | Strongest candidate for deployment after tuning. |

---

# Final Verdict (Short Summary)

| Rank | Model         | Reason |
|------|---------------|--------|
| 1    | XGBoost       | Highest accuracy, stable generalization. |
| 2    | Random Forest | Good accuracy but needs tuning to reduce overfit. |
| 3    | Logistic Regression | Simple, interpretable, dependable. |
| 4    | AdaBoost      | Stable but slightly weaker accuracy. |
| 5    | Decision Tree | Not deployable due to extreme overfitting. |

---